In [1]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

sw_indo = stopwords.words('indonesian') + list(punctuation)

# Import Data

In [3]:
df = pd.read_csv('/kaggle/input/text-dataset/data/kompas.csv')
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


# Extract TFIDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matrix = tfidf.fit_transform(df.teks)

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn(


# TFIDF Similiarity -> Document Similiarity

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856286, 0.00677806,
        0.01513336]])

In [8]:
sim.argsort() # sorting tetapi mengembalikan indeks document

array([[ 932, 1131, 1593, ...,  215,  144,    0]])

In [13]:
print(f"Teks ke-0\t:\n{df.teks[0][:200]}")
print(f"Teks ke-144\t:\n{df.teks[144][:200]}")
print(f"Teks ke-215\t:\n{df.teks[215][:200]}")

Teks ke-0	:
Ginandjar Tetap Ditahan. Jaksa Agung Dilaporkan ke Polri
Kejaksaan Agung memutuskan untuk tetap menahan tersangka kasus korupsi, Ginandjar Kartasasmita, sampai batas waktu yang ditentukan KUHAP. Sedan
Teks ke-144	:
Kejaksaan Agung Terbitkan Surat Penahanan Baru
Kejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita
Teks ke-215	:
Kuasa Hukum Ginandjar Bertahan di Rutan
Sejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag


# Keyword Extraction

In [15]:
vocab = tfidf.get_feature_names_out()
vocab[-10:]

array(['zuniga memilih', 'zunnatul', 'zunnatul mafruhah', 'zurich',
       'zurich northholt', 'zw', 'zw suparman', 'zw tim', 'zx',
       'zx diserbu'], dtype=object)

In [16]:
tfidf_matrix[0].toarray()

array([[0.02115058, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [17]:
sorted_tfidf = tfidf_matrix[0].toarray()[0].argsort()
sorted_tfidf

array([274202, 365455, 365454, ..., 386352, 436624, 169211])

In [18]:
vocab[169211]

'ginandjar'

In [19]:
vocab[436624]

'putusan'

In [20]:
vocab[386352]

'penahanan'

In [21]:
[vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

### Keyword Extraction dalam bentuk function

In [28]:
def extract_keywords_tfidf(doc, tfidf, topk=10):
    matrix = tfidf.transform([doc])
    vocab = tfidf.get_feature_names_out()
    
    sorted_tfidf = matrix[0].toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[-10:])]

In [29]:
text = """"
4 Nilai Plus Ivar Jenner di Timnas Indonesia U-23: Naturalisasi Berkelas, Gelandang Paket Komplet dan Premium

Bola.com, Jakarta Pemain FC Utrecht, Ivar Jenner, tampil istimewa dalam dua pertandingan Timnas Indonesia U-23 di Kualifikasi Piala Asia U-23 2024 di Solo, 9-12 September.

Jenner menjadi starter dalam dua laga itu dan sukses mengantarkan Indonesia maju ke putaran final. Skuad Garuda Muda lolos sebagai juara Grup K setelah menggilas Chinses Taipei 9-0 dan Turkmenistan 2-0.

Pada laga melawan Chinese Taipei, Jenner membuat dua assist untuk gol Witan Sulaeman dan gol kedua Marselino Ferdinand.

Sedangkan pada duel melawan Turkmenistan, Ivar Jenner menjadi pemecah kebuntuan dengan mencetak gol pembuka lewat sepakan jarah jauh.

Dari dua pertandingan ini, terlihat jelas kualitas pemain naturalisasi yang benar-benar dibutuhkan Timnas Indonesia. Ini dia empat nilai plus Ivar Jenner.

Ivar Jenner jago memberikam umpan terukur layaknya pemain level Eropa. Maklum, pesepak bola naturalisasi itu bermain untuk FC Utrecht di kasta teratas Liga Belanda.

Ivar Jenner memang beberapa kali menunjukkan kualitasn operan di atas rata-rata. Passing-passing-nya kerap akurat dan enak untuk diterima rekan setimnya.

 

Akhirnya kita punya pemain yang pintar mengalirkan bola dan tahu posisi teman yang kosong. Masih muda, punya visi dan kemampuan scanning yang amat baik. Tipikal jangkar yang lebih dari apa yang bisa kita impikan untuk Timnas Indonesia.

Pemain jangkar yang dicari, punya long pass bagus, timing sliding dan intersep yang oke.

Arsene Wenger sering menggunakan istilah ‘scanning’ dalam pendekatan sepak bola..

Menurut Wenger, pemain berkelas dunia dibedakan dengan pemain lain karena kepala mereka seperti radar yang terus bergerak.

Mereka sentiasa memahami keadaan di sekitarnya seperti komputer berdasarkan persepsi, pengalaman, dan juga untuk membuat langkah selanjutnya mengambil keputusan.

Kemampuan itu ada di Ivar Jenner. Tak heran, lini tengah Timnas Indonesia U-23 begitu kuat.

Selain mencetak gol, Ivar Jenner juga menjadi penyeimbang di lini tengah. Ia ikut membantu pertahanan ketika Timnas U-23 dalam tekanan.

Ivar Jenner sempat diragukan kualitasnya ketika pertama kali dinaturalisasi. Namun, perlahan tapi pasti, Ivar Jenner mampu membuktikan kualitasnya.

Jenner mampu menjadi pemimpin di lini tengah Timnas Indonesia U-23 pada laga Kualifikasi Piala Asia U-23 2024. Ia pun menciptakan duet yang menarik dengan Arkhan Fikri.

Keduanya memiliki karakter yang mirip. Meski untuk urusan membantu pertahanan Ivar Jenner sedikit lebih baik ketimbang Arkhan.

Melawan Turkmenistan, Ivar Jenner berhasil menyumbang satu gol.

Ivar Jenner menjebol gawang Turkmenistan pada menit ke-40. Sang pemain melepaskan tendangan dari luar kotak penalti yang lebih dulu mengenai pemain Turkmenistan sebelum merobek gawang.
"""

In [30]:
extract_keywords_tfidf(text, tfidf)

['turkmenistan',
 'pemain',
 'gol',
 'naturalisasi',
 'laga',
 'lini',
 'utrecht',
 'taipei',
 'wenger',
 'berkelas']